# Results on Real Data (CHESS/BCC and datasets bootstrapped from Open Images)

In [3]:
from metric import compute_metric_per_dataset, compute_metric_per_layer, random_projection, compute_metric_real
from plot_utils import plot_all_datasets_correlation, plot_dataset_correlation, compute_correlations_real, compute_correlations_from_oi, compute_correlations_from_oi_layers
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from matplotlib.lines import Line2D
import pickle

## Compute results

In [27]:
# Compute map for COCO --> Bootstrapped datasets for different extracted layers
mAP_05_95_list, mAP_50_list, mAP_75_list, AR_1_list, AR_10_list = ([] for i in range(5))
for i in range(1, 21):
    dir = f"/data.nfs/AUTO_TL_OD/ft_model_from_oi/dataset_{i}/checkpoints_vers10/"
    eval_logger = pickle.load(open(dir + 'dataset_1_eval.pkl', "rb"))
    mAP_05_95_list.append(eval_logger['mAP_05_95'])
    mAP_50_list.append(eval_logger['mAP_50'])
    
map_50_from_oi = []
map_95_from_oi = []
for i in range(len(mAP_05_95_list)):
    y =  max(mAP_50_list[i])
    map_50_from_oi.append(y)
    y = max(mAP_05_95_list[i])
    map_95_from_oi.append(y)

    
# Compute metrics fo different layers 

layers = ['layer_5', 'fpn_ms', 'fpn_0', 'fpn_1', 'fpn_2', 'fpn_3', 'fc_7']

dataset_paths = [f"dataset_{i}" for i in range(1,21)]
    
transrate_list, bbox_transrate_list = zip(*[compute_metric_real(metric = 'transrate', dataset_paths = dataset_paths, layer=  layer) for layer in layers])
logme_list, bbox_logme_list = zip(*[compute_metric_real(metric = 'label_LogME', dataset_paths = dataset_paths, layer=  layer) for layer in layers])
xy_logme_list, bbox_xy_logme_list = zip(*[compute_metric_real(metric = 'xy_LogME', dataset_paths = dataset_paths, layer=  layer) for layer in layers])
hscore_list, bbox_hscore_list = zip(*[compute_metric_real(metric = 'hscore', dataset_paths = dataset_paths, layer=  layer) for layer in layers])
regularized_hscore_list, bbox_regularized_hscore_list = zip(*[compute_metric_real(metric = 'regularized_hscore', dataset_paths = dataset_paths, layer=  layer) for layer in layers])

transrate_df, bbox_transrate_df = pd.DataFrame(transrate_list, index = layers), pd.DataFrame(bbox_transrate_list, index = layers)
logme_df, bbox_logme_df = pd.DataFrame(logme_list, index = layers), pd.DataFrame(bbox_logme_list, index = layers)
xy_logme_df, bbox_xy_logme_df = pd.DataFrame(xy_logme_list, index = layers), pd.DataFrame(bbox_xy_logme_list, index = layers)
hscore_df, bbox_hscore_df = pd.DataFrame(hscore_list, index = layers), pd.DataFrame(bbox_hscore_list, index = layers)
regularized_hscore_df, bbox_regularized_hscore_df = pd.DataFrame(regularized_hscore_list, index = layers), pd.DataFrame(bbox_regularized_hscore_list, index = layers)
total_logme_df = pd.DataFrame((logme_df.to_numpy() + xy_logme_df.to_numpy())/2 , index = layers)
bbox_total_logme_df = pd.DataFrame((bbox_logme_df.to_numpy() + bbox_xy_logme_df.to_numpy())/2, index = layers )

map_df =  pd.DataFrame(([map_95_from_oi ]*len(layers)), index = layers)

#Save Data

logme_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_logme_oi_layers.csv')
bbox_logme_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_logme_oi_layers.csv')
xy_logme_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_xy_logme_oi_layers.csv')
bbox_xy_logme_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_xy_logme_oi_layers.csv')
total_logme_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_total_logme_oi_layers.csv')
bbox_total_logme_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_total_logme_oi_layers.csv')
hscore_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_hscore_oi_layers.csv')
bbox_hscore_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_hscore_oi_layers.csv')
regularized_hscore_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_regularized_hscore_oi_layers.csv')
bbox_regularized_hscore_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_regularized_hscore_oi_layers.csv')
transrate_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_transrate_oi_layers.csv')
bbox_transrate_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_transrate_oi_layers.csv')
map_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_map_oi_layers.csv')


In [12]:
# Compute map for COCO --> Bootstrapped datasets for different extracted layers for VISION TRANSFORMER
mAP_05_95_list, mAP_50_list, mAP_75_list, AR_1_list, AR_10_list = ([] for i in range(5))
for i in range(1, 21):
    dir = f"/data.nfs/AUTO_TL_OD/ft_model_from_oi/dataset_{i}/checkpoints_vers20/" #Vers 20 for transformer, 10 for classic faster-rcnn
    eval_logger = pickle.load(open(dir + 'dataset_1_eval.pkl', "rb"))
    mAP_05_95_list.append(eval_logger['mAP_05_95'])
    mAP_50_list.append(eval_logger['mAP_50'])
    
map_50_from_oi = []
map_95_from_oi = []
for i in range(len(mAP_05_95_list)):
    y =  max(mAP_50_list[i])
    map_50_from_oi.append(y)
    y = max(mAP_05_95_list[i])
    map_95_from_oi.append(y)
# Compute metrics fo different layers 

layers = ['layer_5', 'fpn_ms', 'fpn_0', 'fpn_1', 'fpn_2', 'fpn_3', 'penultimate']

dataset_paths = [f"dataset_{i}" for i in range(1,21)]
    
transrate_list, bbox_transrate_list = zip(*[compute_metric_real(metric = 'transrate', dataset_paths = dataset_paths, layer=  layer, data_dir = "/data.nfs/AUTO_TL_OD/extracted_feats/vit/") for layer in layers])
logme_list, bbox_logme_list = zip(*[compute_metric_real(metric = 'label_LogME', dataset_paths = dataset_paths, layer=  layer, data_dir = "/data.nfs/AUTO_TL_OD/extracted_feats/vit/") for layer in layers])
xy_logme_list, bbox_xy_logme_list = zip(*[compute_metric_real(metric = 'xy_LogME', dataset_paths = dataset_paths, layer=  layer, data_dir = "/data.nfs/AUTO_TL_OD/extracted_feats/vit/") for layer in layers])
hscore_list, bbox_hscore_list = zip(*[compute_metric_real(metric = 'hscore', dataset_paths = dataset_paths, layer=  layer, data_dir = "/data.nfs/AUTO_TL_OD/extracted_feats/vit/") for layer in layers])
regularized_hscore_list, bbox_regularized_hscore_list = zip(*[compute_metric_real(metric = 'regularized_hscore', dataset_paths = dataset_paths, layer=  layer, data_dir = "/data.nfs/AUTO_TL_OD/extracted_feats/vit/") for layer in layers])

transrate_df, bbox_transrate_df = pd.DataFrame(transrate_list, index = layers), pd.DataFrame(bbox_transrate_list, index = layers)
logme_df, bbox_logme_df = pd.DataFrame(logme_list, index = layers), pd.DataFrame(bbox_logme_list, index = layers)
xy_logme_df, bbox_xy_logme_df = pd.DataFrame(xy_logme_list, index = layers), pd.DataFrame(bbox_xy_logme_list, index = layers)
hscore_df, bbox_hscore_df = pd.DataFrame(hscore_list, index = layers), pd.DataFrame(bbox_hscore_list, index = layers)
regularized_hscore_df, bbox_regularized_hscore_df = pd.DataFrame(regularized_hscore_list, index = layers), pd.DataFrame(bbox_regularized_hscore_list, index = layers)
total_logme_df = pd.DataFrame((logme_df.to_numpy() + xy_logme_df.to_numpy())/2 , index = layers)
bbox_total_logme_df = pd.DataFrame((bbox_logme_df.to_numpy() + bbox_xy_logme_df.to_numpy())/2, index = layers )

map_df =  pd.DataFrame(([map_95_from_oi ]*len(layers)), index = layers)

#Save Data

logme_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_logme_oi_layers_vit.csv')
bbox_logme_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_logme_oi_layers_vit.csv')
xy_logme_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_xy_logme_oi_layers_vit.csv')
bbox_xy_logme_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_xy_logme_oi_layers_vit.csv')
total_logme_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_total_logme_oi_layers_vit.csv')
bbox_total_logme_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_total_logme_oi_layers_vit.csv')
hscore_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_hscore_oi_layers_vit.csv')
bbox_hscore_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_hscore_oi_layers_vit.csv')
regularized_hscore_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_regularized_hscore_oi_layers_vit.csv')
bbox_regularized_hscore_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_regularized_hscore_oi_layers_vit.csv')
transrate_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_transrate_oi_layers_vit.csv')
bbox_transrate_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_transrate_oi_layers_vit.csv')
map_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_map_oi_layers_vit.csv')


In [4]:
# Compute map for COCO --> BCCD/VOC/CHESS datasets for different extracted layers

map_95= pd.read_csv('/data.nfs/AUTO_TL_OD/results/coco_map_95.csv', header =0, index_col = 0).iloc[:,0].values
layers = ['layer_5', 'fpn_ms', 'fpn_0', 'fpn_1', 'fpn_2', 'fpn_3', 'fc_7']

dataset_paths = ['BCCD', 'CHESS', 'VOC','Global_Wheat', 'Open_Images', ] #Order is important and as to  be similar to the one in map_df
    
transrate_list, bbox_transrate_list = zip(*[compute_metric_real(metric = 'transrate', dataset_paths = dataset_paths, layer=  layer) for layer in layers])
logme_list, bbox_logme_list = zip(*[compute_metric_real(metric = 'label_LogME', dataset_paths = dataset_paths, layer=  layer) for layer in layers])
xy_logme_list, bbox_xy_logme_list = zip(*[compute_metric_real(metric = 'xy_LogME', dataset_paths = dataset_paths, layer=  layer) for layer in layers])
hscore_list, bbox_hscore_list = zip(*[compute_metric_real(metric = 'hscore', dataset_paths = dataset_paths, layer=  layer) for layer in layers])
regularized_hscore_list, bbox_regularized_hscore_list = zip(*[compute_metric_real(metric = 'regularized_hscore', dataset_paths = dataset_paths, layer=  layer) for layer in layers])


transrate_df, bbox_transrate_df = pd.DataFrame(transrate_list, index = layers), pd.DataFrame(bbox_transrate_list, index = layers)
logme_df, bbox_logme_df = pd.DataFrame(logme_list, index = layers), pd.DataFrame(bbox_logme_list, index = layers)
xy_logme_df, bbox_xy_logme_df = pd.DataFrame(xy_logme_list, index = layers), pd.DataFrame(bbox_xy_logme_list, index = layers)
hscore_df, bbox_hscore_df = pd.DataFrame(hscore_list, index = layers), pd.DataFrame(bbox_hscore_list, index = layers)
regularized_hscore_df, bbox_regularized_hscore_df = pd.DataFrame(regularized_hscore_list, index = layers), pd.DataFrame(bbox_regularized_hscore_list, index = layers)
total_logme_df = pd.DataFrame((logme_df.to_numpy() + xy_logme_df.to_numpy())/2 , index = layers)
bbox_total_logme_df = pd.DataFrame((bbox_logme_df.to_numpy() + bbox_xy_logme_df.to_numpy())/2, index = layers )
map_df =  pd.DataFrame(([map_95]*len(layers)), index = layers)


#Save Data
logme_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_logme_layers.csv')
bbox_logme_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_logme_layers.csv')
xy_logme_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_xy_logme_layers.csv')
total_logme_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_total_logme_layers.csv')
bbox_total_logme_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_total_logme_layers.csv')
hscore_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_hscore_layers.csv')
bbox_hscore_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_hscore_layers.csv')
regularized_hscore_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_regularized_hscore_layers.csv')
bbox_regularized_hscore_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_regularized_hscore_layers.csv')
transrate_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_transrate_layers.csv')
bbox_transrate_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_transrate_layers.csv')
map_df.to_csv('/data.nfs/AUTO_TL_OD/results/real_map_layers.csv')

## Correlation for bootstrapped datasets from Open Images
### Study with different layers of extraction

In [12]:
#ResNet backbone
logme_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_logme_oi_layers.csv', header = 0, index_col = 0)
bbox_logme_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_logme_oi_layers.csv', header = 0, index_col = 0)
xy_logme_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_xy_logme_oi_layers.csv', header = 0, index_col = 0)
bbox_xy_logme_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_xy_logme_oi_layers.csv', header = 0, index_col = 0)
total_logme_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_total_logme_oi_layers.csv', header = 0, index_col = 0)
bbox_total_logme_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_total_logme_oi_layers.csv', header = 0, index_col = 0)
hscore_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_hscore_oi_layers.csv', header = 0, index_col = 0)
bbox_hscore_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_hscore_oi_layers.csv', header = 0, index_col = 0)
regularized_hscore_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_regularized_hscore_oi_layers.csv', header = 0, index_col = 0)
bbox_regularized_hscore_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_regularized_hscore_oi_layers.csv', header = 0, index_col = 0)
transrate_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_transrate_oi_layers.csv', header = 0, index_col = 0)
bbox_transrate_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_transrate_oi_layers.csv', header = 0, index_col = 0)
map_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_map_oi_layers.csv', header = 0, index_col= 0)

In [6]:
#VISION TRANSFORMER
logme_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_logme_oi_layers_vit.csv', header = 0, index_col = 0)
bbox_logme_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_logme_oi_layers_vit.csv', header = 0, index_col = 0)
xy_logme_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_xy_logme_oi_layers_vit.csv', header = 0, index_col = 0)
bbox_xy_logme_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_xy_logme_oi_layers_vit.csv', header = 0, index_col = 0)
total_logme_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_total_logme_oi_layers_vit.csv', header = 0, index_col = 0)
bbox_total_logme_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_total_logme_oi_layers_vit.csv', header = 0, index_col = 0)
hscore_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_hscore_oi_layers_vit.csv', header = 0, index_col = 0)
bbox_hscore_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_hscore_oi_layers_vit.csv', header = 0, index_col = 0)
regularized_hscore_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_regularized_hscore_oi_layers_vit.csv', header = 0, index_col = 0)
bbox_regularized_hscore_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_regularized_hscore_oi_layers_vit.csv', header = 0, index_col = 0)
transrate_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_transrate_oi_layers_vit.csv', header = 0, index_col = 0)
bbox_transrate_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_transrate_oi_layers_vit.csv', header = 0, index_col = 0)
map_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_map_oi_layers_vit.csv', header = 0, index_col= 0)

metric_dfs = [logme_df, total_logme_df, hscore_df, regularized_hscore_df, transrate_df]
bbox_metric_dfs = [bbox_logme_df, bbox_total_logme_df, bbox_hscore_df, bbox_regularized_hscore_df, bbox_transrate_df]
metric_names = ['LogME', 'Total LogME', 'H-Score', 'H-Score*', 'TransRate']
df = compute_correlations_from_oi_layers(map_df, metric_dfs, bbox_metric_dfs, metric_names, correlation = 'pearson')
df.to_csv('/data.nfs/AUTO_TL_OD/results/real_oi_summary_layers_vit.csv')
df

Duplicates                                         Bbox  \
                 LogME Total LogME H-Score H-Score* TransRate LogME   
Layer                                                                 
layer_5           0.46        0.45    0.07     0.09      0.27  0.52   
fpn_ms            0.53        0.52    0.13     0.22      0.54  0.56   
fpn_0             0.53        0.51    0.11     0.19      0.51  0.58   
fpn_1             0.53        0.52    0.11     0.20      0.53  0.58   
fpn_2             0.53        0.52    0.12     0.21      0.54  0.58   
fpn_3             0.53        0.52    0.13     0.22      0.54  0.57   
penultimate       0.53        0.52    0.13     0.22      0.54  0.51   

                                                    
            Total LogME H-Score H-Score* TransRate  
Layer                                               
layer_5            0.53   -0.08     0.01      0.45  
fpn_ms             0.55    0.04     0.04      0.50  
fpn_0              0.61    0.00     0.04      0.57  
fpn_1              0.61   -0.00     0.06      0.58  
fpn_2              0.61    0.02     0.08      0.58  
fpn_3              0.62    0.05     0.11      0.58  
penultimate        0.48    0.08     0.08      0.36

### Study of training impact (correlation with varying number of epochs)

In [15]:
#  Get map at different epochs
mAP_05_95_list = []
mAP_05, mAP_10, mAP_20, mAP_30, mAP_40, mAP_50 = ([] for i in range(6))
for i in range(1, 21):
    dir = f"/data.nfs/AUTO_TL_OD/ft_model_from_oi/dataset_{i}/checkpoints_vers10/"
    eval_logger = pickle.load(open(dir + 'dataset_1_eval.pkl', "rb"))
    mAP_05_95_list.append(eval_logger['mAP_05_95'])  

for i in range(len(mAP_05_95_list)):
    map_list = mAP_05_95_list[i]
    mAP_05.append(map_list[4])
    mAP_10.append(map_list[9])
    mAP_20.append(map_list[19])
    mAP_30.append(map_list[29])
    
layer = 'fpn_ms'
#Study for FPN_MS and number of layers
metric_lists = [logme_df.loc[layer], total_logme_df.loc[layer], hscore_df.loc[layer], regularized_hscore_df.loc[layer], transrate_df.loc[layer]]
bbox_metric_lists = [bbox_logme_df.loc[layer], bbox_total_logme_df.loc[layer], bbox_hscore_df.loc[layer], bbox_regularized_hscore_df.loc[layer], bbox_transrate_df.loc[layer]]
metric_names = ['LogME','Total LogME', 'H-Score', 'H-Score*', 'TransRate']
df_05 = compute_correlations_from_oi(mAP_05, metric_lists, bbox_metric_lists, metric_names)
df_10 = compute_correlations_from_oi(mAP_10, metric_lists, bbox_metric_lists, metric_names)
df_20 = compute_correlations_from_oi(mAP_20, metric_lists, bbox_metric_lists, metric_names)
df_30 = compute_correlations_from_oi(mAP_30, metric_lists, bbox_metric_lists, metric_names)

df = pd.concat((df_05, df_10, df_20, df_30), axis = 0)
df.index = ['5 epochs', '10 epochs', '20 epochs', '30 epochs']
df.to_csv('/data.nfs/AUTO_TL_OD/results/real_oi_summary_epochs.csv')
df

Duplicates                                         Bbox              \
               LogME Total LogME H-Score H-Score* TransRate LogME Total LogME   
5 epochs        0.44        0.46    0.25     0.32      0.34  0.46        0.60   
10 epochs       0.31        0.33    0.26     0.30      0.23  0.28        0.45   
20 epochs       0.26        0.30    0.19     0.19      0.19  0.20        0.36   
30 epochs       0.26        0.31    0.20     0.20      0.18  0.11        0.28   

                                      
          H-Score H-Score* TransRate  
5 epochs     0.12     0.09      0.56  
10 epochs    0.07     0.04      0.44  
20 epochs   -0.09    -0.13      0.41  
30 epochs   -0.18    -0.23      0.39

In [9]:
#VISION TRANSFORMER
#  Get map at different epochs
mAP_05_95_list = []
mAP_05, mAP_10, mAP_20, mAP_30, mAP_40, mAP_50 = ([] for i in range(6))
for i in range(1, 21):
    dir = f"/data.nfs/AUTO_TL_OD/ft_model_from_oi/dataset_{i}/checkpoints_vers20/"
    eval_logger = pickle.load(open(dir + 'dataset_1_eval.pkl', "rb"))
    mAP_05_95_list.append(eval_logger['mAP_05_95'])  

for i in range(len(mAP_05_95_list)):
    map_list = mAP_05_95_list[i]
    mAP_05.append(map_list[4])
    mAP_10.append(map_list[9])
    mAP_20.append(map_list[19])
    mAP_30.append(map_list[29])

    
layer = 'fpn_ms'
#Study for FPN_MS and number of layers
metric_lists = [logme_df.loc[layer], total_logme_df.loc[layer], hscore_df.loc[layer], regularized_hscore_df.loc[layer], transrate_df.loc[layer]]
bbox_metric_lists = [bbox_logme_df.loc[layer], bbox_total_logme_df.loc[layer], bbox_hscore_df.loc[layer], bbox_regularized_hscore_df.loc[layer], bbox_transrate_df.loc[layer]]
metric_names = ['LogME','Total LogME', 'H-Score', 'H-Score*', 'TransRate']
df_05 = compute_correlations_from_oi(mAP_05, metric_lists, bbox_metric_lists, metric_names)
df_10 = compute_correlations_from_oi(mAP_10, metric_lists, bbox_metric_lists, metric_names)
df_20 = compute_correlations_from_oi(mAP_20, metric_lists, bbox_metric_lists, metric_names)
df_30 = compute_correlations_from_oi(mAP_30, metric_lists, bbox_metric_lists, metric_names)

df = pd.concat((df_05, df_10, df_20, df_30), axis = 0)
df.index = ['5 epochs', '10 epochs', '20 epochs', '30 epochs']
df.to_csv('/data.nfs/AUTO_TL_OD/results/real_oi_summary_epochs_vit.csv')
df

Duplicates                                         Bbox              \
               LogME Total LogME H-Score H-Score* TransRate LogME Total LogME   
5 epochs        0.45        0.44    0.23     0.34      0.39  0.60        0.59   
10 epochs       0.49        0.49    0.14     0.24      0.47  0.51        0.53   
20 epochs       0.53        0.53    0.11     0.20      0.54  0.53        0.54   
30 epochs       0.39        0.36    0.08     0.22      0.41  0.49        0.46   

                                      
          H-Score H-Score* TransRate  
5 epochs     0.38     0.37      0.36  
10 epochs    0.03     0.02      0.47  
20 epochs   -0.02    -0.02      0.52  
30 epochs    0.11     0.13      0.35

## Correlation for VOC/BCCD/CHESS/...
### Study with different layers of extraction

In [10]:
logme_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_logme_layers.csv', header = 0, index_col = 0)
bbox_logme_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_logme_layers.csv', header = 0, index_col = 0)
xy_logme_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_xy_logme_layers.csv', header = 0, index_col = 0)
bbox_xy_logme_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_xy_logme_layers.csv', header = 0, index_col = 0)
total_logme_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_total_logme_layers.csv', header = 0, index_col = 0)
bbox_total_logme_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_total_logme_layers.csv', header = 0, index_col = 0)
hscore_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_hscore_layers.csv', header = 0, index_col = 0)
bbox_hscore_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_hscore_layers.csv', header = 0, index_col = 0)
regularized_hscore_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_regularized_hscore_layers.csv', header = 0, index_col = 0)
bbox_regularized_hscore_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_regularized_hscore_layers.csv', header = 0, index_col = 0)
transrate_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_transrate_layers.csv', header = 0, index_col = 0)
bbox_transrate_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_bbox_transrate_layers.csv', header = 0, index_col = 0)
map_df = pd.read_csv('/data.nfs/AUTO_TL_OD/results/real_map_layers.csv', header = 0, index_col= 0)

metric_dfs = [logme_df, total_logme_df, hscore_df, regularized_hscore_df, transrate_df]
bbox_metric_dfs = [bbox_logme_df, bbox_total_logme_df, bbox_hscore_df, bbox_regularized_hscore_df, bbox_transrate_df]
metric_names = ['LogME', 'Total LogME', 'H-Score', 'H-Score*', 'TransRate']
df = compute_correlations_from_oi_layers(map_df, metric_dfs, bbox_metric_dfs, metric_names, correlation = 'pearson')
df.to_csv('/data.nfs/AUTO_TL_OD/results/real_summary_layers.csv')
df

Duplicates                                         Bbox              \
             LogME Total LogME H-Score H-Score* TransRate LogME Total LogME   
Layer                                                                         
layer_5      -0.02       -0.02   -0.65    -0.64     -0.75  0.04        0.30   
fpn_ms       -0.00       -0.01   -0.58    -0.54     -0.82  0.10        0.47   
fpn_0         0.02        0.01   -0.57    -0.55     -0.81  0.09        0.43   
fpn_1         0.01        0.00   -0.57    -0.55     -0.82  0.08        0.52   
fpn_2         0.01       -0.00   -0.58    -0.55     -0.82  0.04        0.55   
fpn_3        -0.00       -0.01   -0.58    -0.54     -0.82 -0.08        0.63   
fc_7         -0.00       -0.01   -0.58    -0.54     -0.82  0.13        0.26   

                                    
        H-Score H-Score* TransRate  
Layer                               
layer_5   -0.15    -0.09     -0.35  
fpn_ms    -0.01    -0.03     -0.68  
fpn_0      0.02     0.00     -0.64  
fpn_1      0.02    -0.01     -0.61  
fpn_2     -0.00    -0.03     -0.59  
fpn_3     -0.04    -0.08     -0.57  
fc_7       0.03     0.06     -0.62